# DX 704 Week 7 Project

This week's project will investigate issues in a quadcopter controller based using a linear quadratic regulator.
You will start with an existing model of the system and logs from a quadcopter based on it, investigate discrepancies, and ultimately train a new model of the system dynamics.

The full project description and a template notebook are available on GitHub: [Project 7 Materials](https://github.com/bu-cds-dx704/dx704-project-07).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Introduction

You've just joined a drone startup.
On your first day, you join your new team to watch a test flight for a new quadcopter prototype.
Watching the prototype fly, the team comments that it is not as smooth as usual and suspects that something is off in the controller.
They provide you a copy of the dynamics model and log data from the prototype to investigate.

The quadcopter control model is a slightly more sophisticated version of the 1D quadcopter problem previously considered.

The state vector $\mathbf{x}_t$ now includes an acceleration component, and the action vector now has a servo control for the throttle instead of a raw force component.
\begin{array}{rcl}
\mathbf{x}_t & = & \begin{bmatrix} x_t \\ v_t \\ a_t \end{bmatrix} \\
\mathbf{u_t} & = & \begin{bmatrix} u_t \end{bmatrix}
\end{array}

## Part 1: Reconstruct the Control Matrix

You are provided the dynamics model in the files `model-A.tsv`, `model-B.tsv`, `cost-Q.tsv` and `cost-R.tsv`.
Recompute the control matrix $\mathbf{K}$ to be used in the infinite horizon linear quadratic regulator problem.

In [2]:
%pip install -q control


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# YOUR CHANGES HERE

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import control

# Load the tsv files 
A = pd.read_csv('model-A.tsv', sep='\t', index_col=0).values
B = pd.read_csv('model-B.tsv', sep='\t', index_col=0).values
Q = pd.read_csv('cost-Q.tsv', sep='\t', index_col=0).values
R = pd.read_csv('cost-R.tsv', sep='\t', index_col=0).values


Save $\mathbf{K}$ in a file "control-K-intended.tsv" with columns x, v and a.

In [4]:
# YOUR CHANGES HERE

K, _, _ = control.dlqr(A, B, Q, R)


df_K = pd.DataFrame(K, columns=['x', 'v', 'a'])
df_K.to_csv('control-K-intended.tsv', sep='\t', index=False)

Submit "control-K-intended.tsv" in Gradescope.

## Part 2: Recompute the Actions for the Logged States

You get access to the log data for the prototype as the file "qc-log.tsv".
It conveniently saves all the state and actions made.
Recompute the actions based on your $\mathbf{K}$ matrix computed in part 1.

In [12]:
# YOUR CHANGES HERE


log = pd.read_csv('qc-log.tsv', sep='\t')

# Load from part 1
K = pd.read_csv('control-K-intended.tsv', sep='\t').values

# This is to extract state vector columns x, v, a
X = log[['x', 'v', 'a']].values.T  


u_check = -K @ X  

df_u = pd.DataFrame({
    'index': log.index,
    'u_check': u_check.flatten()
})



Save your computed actions as "qc-check.tsv" with columns "index" and "u_check".

In [11]:
# YOUR CHANGES HERE

df_u.to_csv('qc-check.tsv', sep='\t', index=False)

Submit "qc-check.tsv" in Gradescope.

## Part 3: Reverse Engineer the Actual Control Matrix

Now that you have found a systematic difference between your computed actions and the logged actions, estimate $
$, the control matrix that was actually used to choose actions in the prototype.

Hint: With a linear quadratic regulator, the optimal actions are always linear combinations of the state that are calculated using the control matrix.
You can use linear regression to reverse-engineer the coefficients in the control matrix.

In [14]:
# YOUR CHANGES HERE
log = pd.read_csv('qc-log.tsv', sep='\t')

# Extract state and action values
X = log[['x', 'v', 'a']].values.T  # shape: (3, N)
U = log['u'].values.reshape(1, -1)  # shape: (1, N)

# Solve for -K_actual 
K_actual = -U @ X.T @ np.linalg.inv(X @ X.T)



Save $\mathbf{K}_{\mathrm{actual}}$ in "control-K-actual.tsv" with the same format as "control-K-intended.tsv".

In [ ]:
# YOUR CHANGES HERE

df_K_actual = pd.DataFrame(K_actual, columns=['x', 'v', 'a'])
df_K_actual.to_csv('control-K-actual.tsv', sep='\t', index=False)


Ellipsis

Submit "control-k-actual.tsv" in Gradescope.

## Part 4: Recompute the System Dynamics from the Log Data

On further investigation, it turns out that the control matrix $\mathbf{K}$ in the prototype was modified to compensate for state prediction errors.
You would like to recompute the $\mathbf{A}$ and $\mathbf{B}$ matrices using the log data since they are used to predict the next states.
However, since the action vector $\mathbf{u}_t$ is linearly dependent on the state via $\mathbf{u}_t=-\mathbf{K} \mathbf{x}_t$, you need a new data set so you can separate the effects of the $\mathbf{A}$ and $\mathbf{B}$ matrices.
Your co-workers kindly provide a new file "qc-train.tsv" where noise was added to each action.
Estimate the true $\mathbf{A}$ and $\mathbf{B}$ matrices based on this file.

In [15]:
# YOUR CHANGES HERE

train = pd.read_csv('qc-train.tsv', sep='\t')

X_t = train[['x', 'v', 'a']].values[:-1].T  
U_t = train['u'].values[:-1].reshape(1, -1)  

X_next = train[['x', 'v', 'a']].values[1:].T  

XU = np.vstack([X_t, U_t]) 

AB = X_next @ np.linalg.pinv(XU)  

# Split into A and B
A_new = AB[:, :3]
B_new = AB[:, 3:]

Save $\mathbf{A}$ and $\mathbf{B}$ to "model-A-new.tsv" and "model-B-new.tsv" respectively.

In [16]:
# YOUR CHANGES HERE

pd.DataFrame(A_new, columns=['x', 'v', 'a']).to_csv('model-A-new.tsv', sep='\t', index=False)
pd.DataFrame(B_new, columns=['u']).to_csv('model-B-new.tsv', sep='\t', index=False)

Submit "model-A-new.tsv" and "model-B-new.tsv" in Gradescope.

## Part 5: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 6: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

In [17]:
with open("acknowledgments.txt", "w") as f:
    f.write("""Acknowledgements:
I used the 704 example code for guidance and ChatGPT for proofreading my work.""")